In [18]:
import pandas as pd
import os
import openpyxl
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm, chi2
import matplotlib.pyplot as plt

In [19]:
import pyfolio as pf
from scipy.stats import binom

In [20]:
# Get the current working directory
# current_working_directory = os.getcwd()
# # Load the Excel file
# file_path = '/Users/minhanhle/Documents/MasterArbeit/Eurostoxx1.xlsx'
# sheet_name = 'Tabelle1'
# # Read the Excel file and skip the first 3 rows and row 5 (row 4 after skipping 3)
# df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=[0, 1, 2, 4])
# df.head(3)

In [21]:
# # Remove any columns with missing values
# df = df.dropna(axis=1)
# # Convert the first column to datetime format and set it as the index
# df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0])
# df.set_index(df.columns[0], inplace=True)
# # Filter the data to include only dates between 01.01.2018 and 01.01.2024
# start_date = '2018-01-01'
# end_date = '2024-01-01'
# df = df[(df.index >= start_date) & (df.index <= end_date)]
# # Define the public holidays in Germany from 2018 to 2024
# holidays = [
#     '2018-01-01', '2018-03-30', '2018-04-02', '2018-05-10', '2018-05-21', '2018-10-03', '2018-12-24', '2018-12-25', '2018-12-26', '2018-12-31',
#     '2019-01-01', '2019-04-19', '2019-04-22', '2019-05-30', '2019-06-10', '2019-10-03', '2019-12-24', '2019-12-25', '2019-12-26', '2019-12-31',
#     '2020-01-01', '2020-04-10', '2020-04-13', '2020-05-21', '2020-06-01', '2020-10-03', '2020-12-24', '2020-12-25', '2020-12-26', '2020-12-31',
#     '2021-01-01', '2021-04-02', '2021-04-05', '2021-05-13', '2021-05-24', '2021-10-03', '2021-12-24', '2021-12-25', '2021-12-26', '2021-12-31',
#     '2022-01-01', '2022-04-15', '2022-04-18', '2022-05-26', '2022-06-06', '2022-10-03', '2022-12-24', '2022-12-25', '2022-12-26', '2022-12-31',
#     '2023-01-01', '2023-04-07', '2023-04-10', '2023-05-18', '2023-05-29', '2023-10-03', '2023-12-24', '2023-12-25', '2023-12-26', '2023-12-31',
#     '2024-01-01'
# ]

# # Convert holidays to datetime format and remove them from the DataFrame
# holidays = pd.to_datetime(holidays)
# df = df[~df.index.isin(holidays)]

# Save the cleaned data to a CSV file without row numbers
# output_path = '/Users/minhanhle/Documents/MasterArbeit/MasterArbeitCode/Cleaned_Eurostoxx.csv'
# df.to_csv(output_path, index=True)

In [22]:
df = pd.read_csv('./Cleaned_Eurostoxx.csv')
df.rename(columns={'Name': 'Datum'}, inplace=True)
df.columns = [col.replace(' (~E )', '') for col in df.columns]
df.set_index('Datum', inplace=True)
df.head(5)

,3I GROUP,A P MOLLER MAERSK B,A2A,AAK,AALBERTS,ABB LTD N,ABN AMRO BANK,ABRDN,ACCIONA,ACCOR,...,WHITBREAD,WIENERBERGER,WIHLBORGS FASTIGHETER,WOLTERS KLUWER,WORLDLINE,WPP,YARA INTERNATIONAL,ZALANDO (XET),ZEALAND PHARMA,ZURICH INSURANCE GROUP
Datum,,,,,,,,,,,,,,,,,,,,,
2018-01-02,10.36,1235.13,1.522,11.74,42.29,21.526,26.84,4.76,68.46,43.48,...,38.45,20.56,4.97,43.46,40.64,15.07,38.67,43.95,11.90,253.275
2018-01-03,10.37,1228.24,1.516,11.79,42.80,21.714,26.81,4.71,68.80,43.31,...,38.37,20.74,5.03,43.70,41.14,14.64,39.22,45.19,12.34,252.397
2018-01-04,10.48,1264.70,1.570,11.97,43.51,21.909,27.06,4.77,70.40,43.60,...,38.57,21.08,5.02,44.07,41.98,14.80,39.10,45.10,12.54,253.429
2018-01-05,10.63,1273.07,1.593,12.13,44.13,22.131,27.23,4.78,71.14,43.77,...,38.67,20.82,5.04,44.55,41.78,15.09,39.77,45.14,12.32,255.630
2018-01-08,10.43,1307.33,1.591,12.18,44.33,22.227,27.42,4.86,71.06,44.38,...,38.54,20.90,5.12,44.50,42.36,15.07,40.66,45.31,12.35,257.668


In [23]:
# Berechnung der täglichen Rendite
returns = np.log(df/ df.shift(1))
returns.index = df.index
returns.head(5)

,3I GROUP,A P MOLLER MAERSK B,A2A,AAK,AALBERTS,ABB LTD N,ABN AMRO BANK,ABRDN,ACCIONA,ACCOR,...,WHITBREAD,WIENERBERGER,WIHLBORGS FASTIGHETER,WOLTERS KLUWER,WORLDLINE,WPP,YARA INTERNATIONAL,ZALANDO (XET),ZEALAND PHARMA,ZURICH INSURANCE GROUP
Datum,,,,,,,,,,,,,,,,,,,,,
2018-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,0.000965,-0.005594,-0.003950,0.004250,0.011987,0.008696,-0.001118,-0.010560,0.004954,-0.003918,...,-0.002083,0.008717,0.012000,0.005507,0.012228,-0.028949,0.014123,0.027823,0.036308,-0.003473
2018-01-04,0.010552,0.029253,0.035000,0.015152,0.016453,0.008940,0.009282,0.012658,0.022990,0.006674,...,0.005199,0.016261,-0.001990,0.008431,0.020212,0.010870,-0.003064,-0.001994,0.016078,0.004080
2018-01-05,0.014212,0.006596,0.014543,0.013278,0.014149,0.010082,0.006263,0.002094,0.010457,0.003892,...,0.002589,-0.012411,0.003976,0.010833,-0.004776,0.019405,0.016990,0.000887,-0.017700,0.008647
2018-01-08,-0.018994,0.026556,-0.001256,0.004114,0.004522,0.004328,0.006953,0.016598,-0.001125,0.013840,...,-0.003367,0.003835,0.015748,-0.001123,0.013787,-0.001326,0.022132,0.003759,0.002432,0.007941


In [24]:
# Drop NA
returns = returns.dropna()

#Definiere Gewichte 
gewicht = 1/len(df.columns)

#Definiere Sample/Test Set
sampleSize = len(returns)
windowSize = 250
testWindowSize = sampleSize-windowSize

V = df[windowSize+1:len(df)]*gewicht

In [25]:
#VaR Konfidenzniveau
pVaR = 0.999
#Get Zscores for alpha value
zScore = norm.ppf(pVaR)

#Definiere VaR-Vektoren
normalVaR = np.zeros(testWindowSize)

In [26]:
normalPortfolioSigmas = pd.DataFrame()
normalVaR = pd.DataFrame()
portfolioGewinnVerlust = pd.DataFrame()
for t in range(testWindowSize):
    k = windowSize + t

    # Berechnen der Kovarianzmatrix
    df_test = df[t:k]
    returnsData = returns[t:k]
    covMatrix = np.cov(returnsData, rowvar=False)
    
    # Berechnen der Standardabweichung der Portfolio-Renditen
    v = V.iloc[t, :]
    v_trans = v.T
    sigma = np.sqrt(v@covMatrix@v_trans)
    normalPortfolioSigmas.loc[t,'Sigma'] = sigma

    # Speichern der VaR-Werte
    normalVaR.loc[t,'VaR'] = zScore * sigma
    
    # Speichern der Gewinn-Verlust-Werte
    portfolioGewinnVerlust.loc[t,'GV'] = np.sum(returns.iloc[k, :] * v)

# Plotten der VaR-Werte
# plt.plot()
# plt.plot(range(0, testWindowSize), normalVaR, label='Normal VaR')
# plt.xlabel('Trading Day')
# plt.ylabel('VaR Estimate')
# plt.title('Normal 99% VaR')
# plt.show()
#TODO: x-Achse die Datum angezeigen werden. 

In [27]:
returns_testWindow = returns.iloc[250:]
returns_testWindow.head(3)

,3I GROUP,A P MOLLER MAERSK B,A2A,AAK,AALBERTS,ABB LTD N,ABN AMRO BANK,ABRDN,ACCIONA,ACCOR,...,WHITBREAD,WIENERBERGER,WIHLBORGS FASTIGHETER,WOLTERS KLUWER,WORLDLINE,WPP,YARA INTERNATIONAL,ZALANDO (XET),ZEALAND PHARMA,ZURICH INSURANCE GROUP
Datum,,,,,,,,,,,,,,,,,,,,,
2019-01-02,-0.013699,0.013419,0.019522,0.008333,0.002116,0.003052,0.007824,0.003478,0.035378,0.004381,...,0.011232,0.021979,0.006006,-0.001171,0.000481,-0.003176,0.004482,0.022908,0.008126,0.002999
2019-01-03,-0.016223,-0.017026,0.001869,-0.010008,-0.031124,-0.016430,0.008246,-0.031749,0.020379,-0.016529,...,-0.006862,-0.016439,-0.001998,-0.018530,-0.065019,-0.011733,-0.002986,0.043049,0.015172,-0.004644
2019-01-04,0.046785,0.052058,0.014829,0.019096,0.045810,0.026763,0.029512,0.045542,0.003661,0.023065,...,0.028509,0.030470,0.019803,0.018530,0.017790,0.020181,0.080411,0.032426,0.033961,0.016159


In [28]:
# # Plotten der Daten
# plt.figure(figsize=(10, 6))
# plt.plot(returns_testWindow.index, returns_testWindow.iloc[:, 0])  # Plot der ersten Spalte als Beispiel
# plt.xlabel('Date')
# plt.ylabel('Asset Return')
# plt.title('Asset Returns over Time')

# # Datumswerte auf der x-Achse um 90 Grad drehen
# plt.xticks(rotation=60)

# # Plot anzeigen
# plt.tight_layout()  # Um sicherzustellen, dass die Labels nicht abgeschnitten werden
# plt.show()

In [29]:
num_exceedances = np.sum(portfolioGewinnVerlust < -normalVaR)
print(num_exceedances)
type(num_exceedances)

ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects

In [ ]:
alpha = 0.01
# Calculate Kupiec's Unconditional Coverage Test
def kupiec_test(exceedances, total_obs, alpha):
    """
    Perform Kupiec's Unconditional Coverage Test for VaR backtesting.
    
    Parameters:
    - exceedances: Number of exceedances (exceptions) observed.
    - total_obs: Total number of observations.
    - alpha: Significance level of the test.
    - significance_level: Confidence level for calculating the critical value (default is 0.05).
    
    Returns:
    - p_value: p-value from Kupiec's test.
    """
    expected_exceedances = total_obs * alpha
    failure_rate = num_exceedances / total_obs
    lr_uc = -2 * np.log(((1 - alpha) ** (total_obs - num_exceedances) * (alpha ** num_exceedances)) /
                        ((1 - failure_rate) ** (total_obs - num_exceedances) * (failure_rate ** num_exceedances)))
    
    # p-value from chi-squared distribution with 1 degree of freedom
    p_value = chi2.sf(lr_uc, df=1)
    return lr_uc, p_value

# Perform Kupiec's test

total_obs = len(portfolioGewinnVerlust)
lr_uc, p_value = kupiec_test(num_exceedances, total_obs, alpha)

# Print results
#print(f'Normal VaR at {alpha * 100}% confidence level: {normalVaR:.6f}')
print(f'Number of exceedances: {num_exceedances}')
print(f'Kupiec Test LR (Unconditional Coverage): {lr_uc}')
print(f'Kupiec Test p-value: {p_value}')

Number of exceedances: GV    17
dtype: int64
Kupiec Test LR (Unconditional Coverage): GV    1.357087
dtype: float64
Kupiec Test p-value: [0.24404293]


###Shrinkage + CCM

In [ ]:
df.shape[1]

564

In [30]:
df_test = df.iloc[0:250]
df_test.reset_index(drop=True, inplace=True)

def d_sh(Z):
    T = Z.shape[0]
    N = Z.shape[1]
    S = np.cov(Z, rowvar=False) * (T / (T - 1))

    # Demean data
    Z_0 = Z - np.mean(Z, axis=0)
    P = np.zeros((N, N))

    for i in range(N):
        for j in range(i, N):
            P[i, j] = P[j, i] = np.mean((Z_0[:, i] - Z_0[:, j])**2)

    pi = np.sum(P)

    # Calculate rho
    CCM = np.diag(1 / np.sqrt(np.diag(S)))
    rc = np.mean(CCM)
    CCM1 = np.full((N, N), rc)
    np.fill_diagonal(CCM1, 1)
    
    VCM_shrinkage = np.diag(np.sqrt(np.diag(S))) * CCM1 * np.diag(np.sqrt(np.diag(S)))
    r = np.zeros((N, N))
    np.fill_diagonal(r, np.diag(P))

    for i in range(N):
        for j in range(i + 1, N):
            nu_ii = np.mean((Z_0[:, i] - np.sqrt(S[i, i]))**2)
            nu_jj = np.mean((Z_0[:, j] - np.sqrt(S[j, j]))**2)
            r[i, j] = r[j, i] = (np.sqrt(S[i, j] / S[i, i] * nu_ii + np.sqrt(S[j, i] / S[j, j] * nu_jj)) or rc / 2)

    rho = np.sum(r)

    # Calculation of gamma
    gamma = np.sum((VCM_shrinkage - S)**2)
    kappa = (pi - rho) / gamma
    delta = max(0, min(kappa/T, 1))

    return {'delta': delta, 'S': S, 'VCM_shrinkage': VCM_shrinkage}
    


In [43]:
df_test = df.iloc[0:250]
df_test.reset_index(drop=True, inplace=True)
Z = df_test


In [50]:
T = Z.shape[0]
N = Z.shape[1]
S = np.cov(Z, rowvar=False) * (T / (T - 1))

    # Demean data
Z_0 = Z - np.mean(Z, axis=0)
P = pd.DataFrame()

for i in range(N):
    for j in range(i, N):
        P.iloc[i, j] = P.iloc[j, i] = np.mean((Z_0.iloc[:, i] - Z_0.iloc[:, j])**2)

pi = np.sum(P)

    # Calculate rho
CCM = np.diag(1 / np.sqrt(np.diag(S)))
rc = np.mean(CCM)
CCM1 = np.full((N, N), rc)
np.fill_diagonal(CCM1, 1)
    
VCM_shrinkage = np.diag(np.sqrt(np.diag(S))) * CCM1 * np.diag(np.sqrt(np.diag(S)))
r = np.zeros((N, N))
np.fill_diagonal(r, np.diag(P))

for i in range(N):
    for j in range(i + 1, N):
        nu_ii = np.mean((Z_0[:, i] - np.sqrt(S[i, i]))**2)
        nu_jj = np.mean((Z_0[:, j] - np.sqrt(S[j, j]))**2)
        r[i, j] = r[j, i] = (np.sqrt(S[i, j] / S[i, i] * nu_ii + np.sqrt(S[j, i] / S[j, j] * nu_jj)) or rc / 2)

rho = np.sum(r)

    # Calculation of gamma
gamma = np.sum((VCM_shrinkage - S)**2)
kappa = (pi - rho) / gamma
delta = max(0, min(kappa/T, 1))

print({'delta': delta, 'S': S, 'VCM_shrinkage': VCM_shrinkage})

IndexError: iloc cannot enlarge its target object